In [1]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Auto reload of library
%reload_ext autoreload
%autoreload 2

# Python path
import sys
base_folder = 'DataScience'
location_base = os.path.join(os.getcwd().split(base_folder)[0], base_folder)
location_module = [os.path.join(location_base, 'Module')] 
for each in location_module:
    if each not in sys.path:
        sys.path.append(each)

from import_KK import *
DeviceStrategy_GPU()
from data_KK import *
from description_KK import *
from preprocessing_KK import *
from preprocessing_text_KK import * ##
from algorithm_textmining_KK import * ##
from evaluation_KK import *
from visualization_KK import *

JAVA is in the system path?:  False
JAVA is in the system path?:  Adding...
Operation Machine:  Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
Operation Platform:  64bit
OS Type:  Windows
OS Version:  10
Python Version:  3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
cannot import name 'DiagnosticOptions' from 'torch.onnx._internal.exporter' (C:\Users\KK\anaconda3\envs\py310cuda118\lib\site-packages\torch\onnx\_internal\exporter\__init__.py)

In [2]:
def get_data_from_path(folder_location, folder_name=False, concat_axis='row', 
                       reset_index=False, date_column=None):
    # path_folder 하위의 모든 폴더위치와 내부 file 출력
    df = pd.DataFrame()
    print('Getting data from', len(os.listdir(folder_location)), 'folders...')
    for (path, dir, files) in os.walk(folder_location):
#         print(path)
        for file in tqdm(files):
            path_file = os.path.join(path, file)

            ## 데이터 로딩
            if path_file[-4:] == 'xlsx':
                df_sub = pd.read_excel(path_file)
            elif path_file[-3:] == 'csv':
                df_sub = pd.read_csv(path_file)
                
            ### 왓차피디어 변수명변경
            if 'reviewDate' in df_sub.columns:
                df_sub.rename(columns={'year':'date'}, inplace=True)
            elif 'writer' in df_sub.columns:
                df_sub.rename(columns={'writer':'username', 'comment':'review'}, inplace=True)

            ## 키워드 태깅 여부
            if folder_name:
                df_sub['Folder_Name'] = file 
                df_sub['Opening'] = os.path.basename(path).split('(')[1].split(')')[0]
                           
            ## 결합
            if concat_axis == 'col':
                df = pd.concat([df, df_sub], axis=1)
            elif concat_axis == 'row':
                df = pd.concat([df, df_sub], axis=0)
    
    # 정리
    if reset_index:
        df = df.reset_index().iloc[:,1:]
    if date_column != None:
        df[date_column] = pd.to_datetime(df[date_column])
                
    return df

# Hyperparameters

In [3]:
%matplotlib inline
# 데이터 파라미터
TARGET = 'Comment'
CATEGORY = 'Opening'
# 전처리모델 파라미터
STOPWORDS = ['정보 없음', '없음', '정보', '영화', '너무', '정말']
NGRAM_RANGE = (1,3)
TFIDF_MAXCOL = 5000
TFIDF_DELLOWFREQ = False
KEYBERT_TOPNKWD = 5
NUM_SHOWKEYWORD = 100
# 모델링 파라미터
TOKENIZER = 'KeyBERT'
# euclidean, manhattan, chebyshev, minkowski, canberra, braycurtis, mahalanobis, wminkowski, seuclidean, cosine, correlation
# haversine, hamming, jaccard, dice, russelrao, kulsinski, ll_dirichlet, hellinger, rogerstanimoto, sokalmichener, sokalsneath, yule
UMAP_METRIC = 'euclidean'
UMAP_SEED = 123
NUM_TOPICS = 'auto'
NUM_TOPICWORDS = 10
NUM_TOPICSAMPLES = 5
# 저장
SAVE_LOCAL = True
SAVE_NAME_PREP = 'wordfreq'
SAVE_NAME_LDA = 'Topics_byTokenLDA.xlsx'
SAVE_NAME_BERTopic = 'Topics_byBERTopic.xlsx'

# Data & Preprocessing

In [4]:
# 데이터
## 로딩
df = get_data_from_path(os.path.join(os.getcwd(), 'Data_Raw'), folder_name=True, concat_axis='row', 
                        reset_index=True, date_column=None)
## 변수명 변환
df.rename(columns={'Folder_Name':'Movie', 'username':'User', 'date':'Viewing', 'review':'Comment', 'rating':'Rating', 
                   'likes':'Like', 'dislikes':'Dislike'}, inplace=True)
## 전처리
df.Viewing = df.Viewing.apply(lambda x: str(x)[:4])
df['Source'] = df.Movie.apply(lambda x: x.split('(')[1].split(')')[0])
df.Source.replace({'네티즌':'Naver(Netizen)', '실관람객':'Naver(Audience)'}, inplace=True)
df.Source = df.Source.apply(lambda x: 'Watcha' if len(x) == 4 else x)
df.Movie = df.Movie.apply(lambda x: x.split('(')[0])
## 삭제
df.drop(columns=['url', 'reviewDate'], inplace=True)
## 결측치처리
df_sub = df[~df.Like.isnull()].fillna(0)
df = df.combine_first(df_sub)
## 정리
df = df[['Movie', 'Opening', 'Source', 'User', 'Viewing', 'Rating', 'Like', 'Dislike', 'Comment']]
df

Getting data from 35 folders...


0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 57.74it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 41.79it/s]


,Movie,Opening,Source,User,Viewing,Rating,Like,Dislike,Comment
0,그대 어이가리,2021,Watcha,차칸수니리,2022,2.0000,13,0.0000,배우들 연기가 어색하게 보이는 것은 배우들 문제가 아니다. 이야기 흐름이 자연스럽지 않은것이 단번에 느껴진다. 감독의 실수가 너무 크다.
1,그대 어이가리,2021,Watcha,Taeseung Kim,2022,3.0000,2,0.0000,살아남은 보단 남겨진 이들에게
2,그대 어이가리,2021,Watcha,イチ,2022,1.0000,2,0.0000,국악을 알리는건 좋은데 왤케 피곤하지... 쓸데없는 장면 너무많고 필요없는연출너무많고 연기력은 너무부족하고 최악의 영화에 시간을 왤케 길고 보는게 진짜 고역이었다 지금세대가 이해할수없는 늙은 감성 내아까운2시간 어이가리
3,그대 어이가리,2021,Watcha,각시탈,2024,3.5000,1,0.0000,"“어이가리”의 어이에는 세가지 정도의 뜻이 있다. 첫째는 “왜?”의 어이이고, 둘째는 ”어떻게“의 어이이며, 셋째는 ”어디를“의 어이이다. 영화에는 이 세가지 의미가 다 담겨 있다. 영화의 연출이나 배우들의 연기를 굳이 논하고 싶지 않다. 다만, 절대로 나이 드신 부모님(특히, 어머니)과 같이 보지는 않기를 추천 한다. 영화가 더 이상 영화가 아니라 우리에게, 나에게 닥칠 현실이기 때문이다. 현실은 영화와는 비교도 안될 정도로 더욱 힘들고 괴로울 것이라는 걸 우리는 어렴풋이나마 알고 있기 때문이다. . . . 나이가 드니 테스토스테론 보다 에스트로겐이 더 많이 분비되는 것이 확실하다. 눈물 참느라 힘들었다."
4,그대 어이가리,2021,Watcha,후루케가뜨,2023,2.0000,1,0.0000,실제 비슷한 생활을 해본 입장에서 조금은 어설프기도 한 공감하기 힘든 지루한 진행. 자꾸 그걸 느끼라고 하는데 자꾸 안 느껴지는..
...,...,...,...,...,...,...,...,...,...
190575,카시오페아,2022,Naver(Audience),woln****,2024,7,0,0.0000,알츠하이머의 무서움을 느꼈습니다.
190576,카시오페아,2022,Naver(Audience),play****,2022,6,4,5.0000,볼만합니다 재밌었어요.
190577,카시오페아,2022,Naver(Audience),dudw****,2022,10,16,21.0000,너무재미있어요ㅋㅋㅋㅋ
190578,카시오페아,2022,Naver(Audience),swdi****,2022,6,12,23.0000,"배우들 연기 좋았어요. 그런데 뭐랄까 너무 잘하려고 하는 느낌이 살짝 들어서 그 부분이 아쉬었던 듯....연출도 약간 매끄럽지 않은 느낌이고, 하지만 잔잔하게 흘러가는 영상들은 참 좋았어요....앞으로 감독님의 발전된 모습 더욱 응원하게 됩니다!"


In [5]:
## 텍스트 전처리
df[TARGET+'_Prep'] = df[TARGET].progress_apply(lambda x: text_preprocessor(x, del_number=False, 
                                                                           del_bracket_content=False,
                                                                           stop_words=STOPWORDS))
## 무의견 삭제
df.dropna(subset=[TARGET], inplace=True)
df = df[df[TARGET] != '']
df = df.reset_index().iloc[:,1:]
df.to_csv(os.path.join(os.getcwd(), 'Data', 'df_movie.csv'), encoding='utf-8-sig')

100%|████████████████████████████████████████████████████████████████████████| 190580/190580 [00:56<00:00, 3346.14it/s]


In [7]:
## 단어빈도 추출과 변환
# word_freq, wordadj_freq, df = preprocessing_wordfreq(df, colname_target=TARGET, colname_category=None, 
#                                                      ngram_range=NGRAM_RANGE,
#                                                      tfidf_maxcol=TFIDF_MAXCOL, tfidf_dellowfreq=TFIDF_DELLOWFREQ,
#                                                      keybert_topnkwd=KEYBERT_TOPNKWD,
#                                                      num_showkeyword=NUM_SHOWKEYWORD, 
#                                                      save_local=SAVE_LOCAL, save_name=SAVE_NAME_PREP)
word_freq_categ, wordadj_freq_categ, df_freq = preprocessing_wordfreq(df, colname_target=TARGET, colname_category=CATEGORY,
                                                                ngram_range=NGRAM_RANGE,
                                                                tfidf_maxcol=TFIDF_MAXCOL, tfidf_dellowfreq=TFIDF_DELLOWFREQ,
                                                                keybert_topnkwd=KEYBERT_TOPNKWD,
                                                                num_showkeyword=NUM_SHOWKEYWORD, 
                                                                save_local=SAVE_LOCAL, save_name=SAVE_NAME_PREP)

  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

training was done. used memory 2.024 Gby 2.024 Gb
all cohesion probabilities was computed. # words = 1348
all branching entropies was computed # words = 12495
all accessor variety was computed # words = 12495
C:/Users/KK/anaconda3/Lib/site-packages/soynlp
cannot access local variable 'f' where it is not associated with a value
cannot access local variable 'f' where it is not associated with a value
before postprocessing 1720
_noun_scores_ 419
after postprocessing 224


  5%|███▉                                                                               | 1/21 [00:17<05:54, 17.71s/it]

training was done. used memory 2.053 Gbory 2.040 Gb
all cohesion probabilities was computed. # words = 11762
all branching entropies was computed # words = 71396
all accessor variety was computed # words = 71396
C:/Users/KK/anaconda3/Lib/site-packages/soynlp
cannot access local variable 'f' where it is not associated with a value
cannot access local variable 'f' where it is not associated with a value
before postprocessing 12916
_noun_scores_ 2376
after postprocessing 1187


 10%|███████▊                                                                          | 2/21 [04:14<46:24, 146.56s/it]

...

training was done. used memory 1.296 Gbry 1.294 Gb
all cohesion probabilities was computed. # words = 4477
all branching entropies was computed # words = 29584
all accessor variety was computed # words = 29584
C:/Users/KK/anaconda3/Lib/site-packages/soynlp
cannot access local variable 'f' where it is not associated with a value
cannot access local variable 'f' where it is not associated with a value
before postprocessing 4976
_noun_scores_ 895
after postprocessing 466


 95%|█████████████████████████████████████████████████████████████████████████████▏   | 20/21 [50:35<02:31, 151.34s/it]

training was done. used memory 1.334 Gbry 1.330 Gb
all cohesion probabilities was computed. # words = 6482
all branching entropies was computed # words = 39477
all accessor variety was computed # words = 39477
C:/Users/KK/anaconda3/Lib/site-packages/soynlp
cannot access local variable 'f' where it is not associated with a value
cannot access local variable 'f' where it is not associated with a value
before postprocessing 7434
_noun_scores_ 1562
after postprocessing 807


100%|█████████████████████████████████████████████████████████████████████████████████| 21/21 [52:15<00:00, 149.29s/it]


KeyboardInterrupt: 

# Descriptive

In [4]:
# 빈도분석
table_valuecounts(df, save_local=True)

,0,1,2,3,4,5,6,7,8,9
Feature,Title,Title,Title,Title,Title,Title,Title,Title,Title,Title
Value,노트북,스틸 앨리스,내일의 기억,첫키스만 50번째,메멘토,더 파더,카시오페아,내 머리속의 지우개,그대를 사랑합니다,친정엄마
Count,599,592,507,470,397,357,315,311,296,296
Ratio,10.9466,10.8187,9.2654,8.5892,7.2551,6.5241,5.7566,5.6835,5.4094,5.4094


,0,1,2,3,4,5,6,7,8,9
Feature,User,User,User,User,User,User,User,User,User,User
Value,love****,jung****,blue****,moon****,shin****,youn****,kang****,park****,happ****,kims****
Count,27,19,18,15,14,12,12,11,10,10
Ratio,0.4934,0.3472,0.3289,0.2741,0.2558,0.2193,0.2193,0.2010,0.1827,0.1827


,0,1,2,3,4,5,6,7,8,9
Feature,Date,Date,Date,Date,Date,Date,Date,Date,Date,Date
Value,2022-06-02 15:59:00,2023-01-23 20:43:00,2013-07-12 17:25:00,2024-08-26 11:03:00,2018-09-22 02:36:00,2023-01-23 20:47:00,2019-03-18 00:39:00,2013-07-12 22:38:00,2021-04-27 10:51:00,2013-07-12 10:17:00
Count,5,5,3,3,3,3,2,2,2,2
Ratio,0.0914,0.0914,0.0548,0.0548,0.0548,0.0548,0.0365,0.0365,0.0365,0.0365


,0,1,2,3,4,5,6,7,8,9
Feature,Rating,Rating,Rating,Rating,Rating,Rating,Rating,Rating,Rating,Rating
Value,10,9,8,1,7,6,5,2,4,3
Count,3511,559,553,260,239,171,65,60,37,17
Ratio,64.1630,10.2156,10.1060,4.7515,4.3677,3.1250,1.1879,1.0965,0.6762,0.3107


,0,1,2,3,4,5,6,7,8,9
Feature,Comment,Comment,Comment,Comment,Comment,Comment,Comment,Comment,Comment,Comment
Value,감동과 꼭 봐야 한다 배우들 연기 짱 이다,날아라,아름답다,좋아요,감동적이네요,영화를 본지 반년이 지나가는데 여운이 남고 장면 장면들이 기억에 남아 마음이 아파요 추천합니다,마음이 따뜻해지는,크리스토퍼 놀란은 천재다,실제로 일어날수있는 내용들이라 집중이 되었던거 같아요 눈물 콧물 엄청 흘렸네요삶이란 많은 생각을 하게 되는것 같아요,엄마 사랑해요
Count,4,3,3,3,3,2,2,2,2,2
Ratio,0.0731,0.0548,0.0548,0.0548,0.0548,0.0365,0.0365,0.0365,0.0365,0.0365


...

,0,1,2,3,4,5,6,7,8,9
Feature,Year,Year,Year,Year,Year,Year,Year,Year,Year,Year
Value,2021,2015,2016,2017,2013,2022,2018,2023,2020,2014
Count,886,587,525,508,442,364,356,294,271,260
Ratio,16.1915,10.7273,9.5943,9.2836,8.0775,6.6520,6.5058,5.3728,4.9525,4.7515


,0,1,2,3,4,5,6,7,8,9
Feature,Month,Month,Month,Month,Month,Month,Month,Month,Month,Month
Value,5,4,6,7,11,3,10,9,8,12
Count,700,689,596,547,473,458,448,351,321,318
Ratio,12.7924,12.5914,10.8918,9.9963,8.6440,8.3699,8.1871,6.4145,5.8662,5.8114


,0,1,2,3,4,5,6,7,8,9
Feature,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP,Token_SoyNLP
Value,,인생,눈물,엄마,연기,재미,감동적,아름,기억,가슴
Count,895,70,61,58,57,51,49,44,41,37
Ratio,16.3560,1.2792,1.1148,1.0599,1.0417,0.9320,0.8955,0.8041,0.7493,0.6762


,0,1,2,3,4,5,6,7,8,9
Feature,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF,Token_TF-IDF
Value,,진짜,감동,연기,사랑,보고,최고의,재미,엄마,아름다운
Count,60,19,19,10,9,9,9,8,8,7
Ratio,1.0965,0.3472,0.3472,0.1827,0.1645,0.1645,0.1645,0.1462,0.1462,0.1279


,0,1,2,3,4,5,6,7,8,9
Feature,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT,Token_KeyBERT
Value,,진짜,기억,감동,눈물이,영화가,연기,사랑,평점,보고
Count,44,17,15,13,10,9,9,8,8,8
Ratio,0.8041,0.3107,0.2741,0.2376,0.1827,0.1645,0.1645,0.1462,0.1462,0.1462


In [5]:
# 문장 토큰 길이 분포
plot_histogram_senttoken(df['Token_'+TOKENIZER])    

,count,mean,std,min,25%,50%,75%,max
Length,"5,472.0000",4.1376,3.0049,1.0000,2.0000,3.0000,5.0000,22.0000


In [6]:
# 연도별 의견수 분포 분포
plot_histogram(df[['Year']])

,count,mean,std,min,25%,50%,75%,max
Year,"5,472.0000","2,016.9781",4.3391,"2,003.0000","2,015.0000","2,017.0000","2,021.0000","2,024.0000"


In [7]:
# 전체 주요 키워드 분포
wf_colnames = [col for col in word_freq.columns if col.find(TOKENIZER) != -1]
plot_donut_wordfreq(word_freq[wf_colnames])

In [8]:
# 연도별 주요 키워드 분포
wfc_colnames = [col for col in word_freq_categ.columns if col.find(TOKENIZER) != -1]
plot_treemap_wordfreq(word_freq_categ[wfc_colnames])

In [9]:
# 단어 벡터화 및 시각화

# Modeling

## LDA

- Latent Dirichlet Allocation (LDA)는 토픽모델링에 이용되는 대표적인 알고리즘

> - 토픽 모델링이 제공하는 토픽은 어떤 주제를 구성하는 단어들인지로 정의되며,
> - 문서 집합에서 이 단어 집합을 찾아 우리의 문서가 어떤 토픽인지 알 수 있음

- LDA 모델을 학습하기 전에 몇 개의 토픽으로 나눌 것인지 먼저 결정해야 함

> - 일반적으로 (1) perplexity와 coherence를 함께 고려하여 결정 + (2) 정성적인 토픽 갯수 고려하여 결정
>> - perplexity: 모델이 얼마나 잘 예측하는지 나타내는 지표로, 낮은 perplexity 값은 모델이 문서에서 나타나는 단어를 더 잘 예측한다는 것을 의미
>> - coherence: 발견된 토픽이 얼마나 의미있는지 나타내는 지표로, 높은 coherence 값은 토픽 내 단어들이 서로 관련성이 높다는 것을 의미

- LDAvis

> (1) Intertopic Distance Map (via multidimensional scaling)
> - 각 토픽이 가진 고차원의 단어들을 PCA로 2차원 축소 시각화
> - 원크기: 토픽의 단어들이 얼마나 포함 및 분포되어 있는지
> - 원거리: 토픽간의 유사성 (2개의 원이 겹친다면 유사한 토픽이라는 의미)
>
> (2) Top-30 Most Relevant Terms for Topic
> - 토픽 별로 가장 자주 등장하는 단어들(람디=1)과 토픽 간 차이가 많이나는 단어들(람다=0)을 종합적으로 고려 

In [19]:
topic_kw_LDA, topic_doc_LDA, \
doc_topic_LDA, model_LDA = modeling_LDA(df['Token_'+TOKENIZER], df['Comment'],
                                        num_topics=NUM_TOPICS, num_topicwords=NUM_TOPICWORDS, num_topicsamples=NUM_TOPICSAMPLES,
                                        save_local=SAVE_LOCAL, save_name=SAVE_NAME_LDA)
display(topic_kw_LDA, topic_doc_LDA, doc_topic_LDA)

2024-09-10 22:48:42,721	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Optimal Topic Number:  2


[Kss]: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


,Related Keywords,Number of Sentences,Percentage
Topic 1,"'사랑(21.3%)', '보고(14.4%)', '연기(9.6%)', '있는(9.0%)', '눈물(8.5%)', '감동(8.0%)', '배우(8.0%)', '영화를(7.4%)...",2831,0.5174
Topic 2,"'연기(24.4%)', '엄마(15.2%)', '진짜(11.1%)', '기억(9.7%)', '감동(9.7%)', '보는(8.3%)', '마지막(6.5%)', '치매(5.5%...",2641,0.4826


,Topic Number,Weight,Comment
3961,Topic 1,0.9642,에휴 난생 영화보면서 울게만든 영화는 진짜 이게 처음이다아직도 여운이 남고 다시는 보고싶지 않는영화다너무 감동깊고 자꾸 어머니가 생각나고 어머니 걱정이많이 나게 하는 영화다...
4,Topic 1,0.9624,아름다운 영화울기도 했지만 가족의 소중함을 있었던 영화입니다 치매 가족의 현실을 보여줘서 슬프지만 가족의 사랑 담아낸 영화라고 생각합니다 슬프지만 아름다운 영화제에서 51관...
2798,Topic 1,0.9620,최고의 로맨스 삶을 관통하는 진리를 다룬 영화는 아니지만 킬링타임으로는 최고 영화는 주기적으로 봐줘야함 사랑을 했다면 죽기 전에 삶에 대해 후회가 없을 연애가 아닌 사랑을 ...
3891,Topic 1,0.9606,제목 누가 지었나 영화는 제목을 잘못 지었다 나는 영화를 볼 기회가 몇번은 있었지만 제목만 보고 단순 신파극이라 생각하고 보지 않았는데 오늘 보게 되었다 영화보면서 이렇게 ...
4360,Topic 1,0.9601,이리 눈물이 나는지 영화는 울라고 강요하지 않는데도 마음이 먹먹하면서 눈물이 멈추지 않네요 항상 집안일을 도와주시는 시어머니와 시어머니 친구분들과 봤는데 저도 울고 시어머니...
2499,Topic 2,0.9626,서예지 논란 안보다가 스릴러가 땡겨서 그냥 보게됬는데 진짜 최근에 본 스릴러 중 가장 재밌다 반전의 반전이다 사실 처음부터 김강우가 남편이 아니고 살인자 같다는 느낌이 들긴...
89,Topic 2,0.9552,우와 진짜 스토리도 좋고 긴장감도 높아서 몰입해서 보느라 시간이 되게 빨리 갔어요 반전의 반전의 반전의 반전의 영화예요 마지막까지 긴장 놓치기 금지 김강우 배우님 존잘
2150,Topic 2,0.9536,진짜 명작중의 명작이네클래식만큼이나 걸작임짱임그 정도로 재밌게 본 영화이기도 하고기억을 잃어가는 주인공을 소재로 나를 잊지 말아요가 졸작인지 생각해보면 영화가 수작인지 알수...
2872,Topic 2,0.9528,연출도 연기도 미쳤다는 밖엔 특히 백전노장 안소니 홉킨스의 다양한 표정 연기는 일품 마지막 씬에서 팔순을 넘긴 노인이 엄마를 그리워하는 어린아이의 감정을 전달하는 연기엔 전...
2988,Topic 2,0.9513,앤소니 홉킨스 연기는 진짜 눈빛에서 모든 감정이 다 느껴진다 솔직히 내용 자체만 보면 별 내용이 없지만 누가 별로라고 할 있을까 여운이 오래가는 영화임 잎사귀가 떨어지고 있...


,Comment,Related Topic,Weight,Each Topic Weights
0,29년 인생에 이렇게 여운이 진하게 남는 영화는 처음이에요 로맨스나 드라마 장르만 보면 코를 고는 저였지만 영화만큼은 처음부터 끝까지 몰입해서 봤습니다 연출과 ost가 눈물...,1,0.8180,"[(0, 0.81796235), (1, 0.18203765)]"
1,지금이순간 무엇을 선택해야 삶에 끝에서 후회가 적을까요아이러니 하게도 죽음을 깊이 생각하니 지금 이순간 좋은 삶의 선택이 보이는듯 합니다 강추합니다,2,0.8850,"[(0, 0.1150399), (1, 0.8849601)]"
2,지인이 괜찮다고 해서 기대감 없이 봤는데 생각보다 재밌어서 놀랐네요 여운이 오래 남아서 아직도 생각나네요,1,0.6128,"[(0, 0.61283934), (1, 0.3871607)]"
3,치매 걸린 가족들이 겪어야 하는 리얼한 얘기였습니다,2,0.5997,"[(0, 0.40033138), (1, 0.5996686)]"
4,아름다운 영화울기도 했지만 가족의 소중함을 있었던 영화입니다 치매 가족의 현실을 보여줘서 슬프지만 가족의 사랑 담아낸 영화라고 생각합니다 슬프지만 아름다운 영화제에서 51관...,1,0.9624,"[(0, 0.9623803), (1, 0.037619684)]"
...,...,...,...,...
5467,많은걸 베웠습니다,2,0.8275,"[(0, 0.17254311), (1, 0.8274569)]"
5468,진짜 흥행 안한건지 모르겠음 원래부터 서현진 배우님 팬이라서 영화도 기대 하고 봤지만 기대 이상으로 감동적이고 무엇보다 서현진배우님 연기에 한번 놀라고 감동받은 작품임 이거...,1,0.8618,"[(0, 0.86175877), (1, 0.13824123)]"
5469,배우들 연기력 하나만으로 몰입갑 쩌는,2,0.8217,"[(0, 0.1783147), (1, 0.82168525)]"
5470,서현진배우는 최고의 배우 입니다사랑합니다,1,0.8789,"[(0, 0.8788716), (1, 0.12112841)]"


## BERTopic

In [20]:
topic_kw_BERT, topic_doc_BERT, \
doc_topic_BERT, model_BERT = modeling_BERTopic(df['Comment'], vectorizer_type='count', 
                                               ngram_range=NGRAM_RANGE,
                                               tfidf_maxcol=TFIDF_MAXCOL, 
                                               umap_metric=UMAP_METRIC, umap_randomseed=UMAP_SEED,
                                               num_topics=NUM_TOPICS, num_topicwords=NUM_TOPICWORDS, num_topicsamples=NUM_TOPICSAMPLES,
                                               save_local=SAVE_LOCAL, save_name=SAVE_NAME_BERTopic)
display(topic_kw_BERT, topic_doc_BERT, doc_topic_BERT)

Valid metrics for dimension reduction of embeddings...:

    * euclidean
    * manhattan
    * chebyshev
    * minkowski
    * canberra
    * braycurtis
    * mahalanobis
    * wminkowski
    * seuclidean
    * cosine
    * correlation
    * haversine
    * hamming
    * jaccard
    * dice
    * russelrao
    * kulsinski
    * ll_dirichlet
    * hellinger
    * rogerstanimoto
    * sokalmichener
    * sokalsneath
    * yule


,Related Keywords,Number of Sentences,Percentage,Representative_Docs
Topic 1,"'진짜(13.7%)', '보고(12.0%)', '눈물이(10.9%)', '있는(10.4%)', '다시(10.2%)', '연기(10.0%)', '마지막(8.4%)', '좋은(...",2461,0.4497,"[7남매를 홀로 키워내신 엄마 생각나서 보는 내내 울었습니다 진한 감동과 여운이 오래도록 남는 좋은 영화입니다, 내가 이상한걸까이 영화는 보고 보고 한번 봐도 볼 마다 느낌..."
Topic 2,"'진짜(13.6%)', '영화를(13.2%)', '연기(10.6%)', '보고(10.0%)', '있는(9.4%)', '다시(9.3%)', '좋은(8.8%)', '영화가(8....",2417,0.4417,"[역시 줄리안 무어 최고의 연기 최고의, 영화를 작년에 보고 어제 추석특집이라면서 무료영화로 풀렸길래 봤는데 때보다 재밌다 기타치면서 노래부르는 씬은 진짜 명장면 이렇게 몰..."
Topic 3,"'눈물이(13.6%)', '엄마가(11.9%)', '영화를(11.2%)', '그냥(10.4%)', '울면서(9.5%)', '울었다(9.3%)', '울고(9.0%)', '정도...",120,0.0219,[딸 아이를 둔 엄마의 입장으로 영화를 보니까 영화관에서 오열할 정도로 슬프고 마음이 힘들었어요 내가 낳은 사랑스러운 내 아이를 기억하지 못한다는 사실을 받아들일 있는 엄마...
Topic 4,"'이기적인(11.6%)', '남자를(11.3%)', '로맨스(10.7%)', '결혼(10.7%)', '없는(10.6%)', '아름다운(10.3%)', '노아와(9.2%)',...",70,0.0128,[라이언 고슬링 레이첼 맥아담스의 찬란했던 리즈 시절을 보는 것만으로 만족 근데 내가 보수적이여서 그런가 내일 모레 결혼 하는 여자가 결혼 할 남자 버리고 바람 아닌 바람 ...
Topic 5,"'슬프다(16.5%)', '슬프고(13.2%)', '슬픔(12.0%)', '슬픈영화(11.8%)', '영화입니다(9.8%)', '슬픈(8.6%)', '공포스럽다(7.7%)'...",53,0.0097,"[아너무 슬프다, 슬프다, 슬프다]"
Topic 6,"'최고의(18.5%)', '영화입니다(12.7%)', '영화다(12.0%)', '가치가(9.5%)', '영화였습니다(9.0%)', '아름다운(8.1%)', '볼만한(7.9%...",51,0.0093,"[올해 최고의 영화다, 인생 최고의 영화입니다, 언제봐도 최고의 영화다]"
Topic 7,"'선생님(15.6%)', '최고(13.3%)', '김강우(10.6%)', '한국(9.9%)', '사람들때문에(9.0%)', '안어울림(9.0%)', '연기력(8.5%)', ...",39,0.0071,"[김강우 배우의 연기력 끝까지 긴장하며 볼 있었다, 순재 선생님 최고 송재호 선생님 최고 김수미 선생님 최고 송이뿐선생님 최고, 서예지역할에 전혀 안어울림 서예지와 김강우 ..."
Topic 8,"'눈물이(24.3%)', '울었습니다(10.9%)', '울었다(9.6%)', '멈추지(9.4%)', '나는(9.3%)', '보면서(8.4%)', '마지막(7.6%)', '울...",37,0.0068,"[슬픔을 강요하지 않는데도 마음이 아릿해지며 눈물이 계속 흘렀다, 이걸 보면서 어머니계 잘해야겠다고 생각한 순간 코끝이 찡해지며 눈물이 나왔다, 언제봐도 지금봐도 나중에봐도..."
Topic 9,"'무섭다(22.7%)', '두려움(16.5%)', '두렵고(11.0%)', '못해(10.1%)', '공포(8.9%)', '보는내내(6.9%)', '기억을(6.4%)', '1...",36,0.0066,"[발상의 전환 이것이 진짜 호러다 무섭다, 현실이 스릴러보다 무섭다, 알지 못해 두렵고 인지하지 못해 무섭다]"
Topic 10,"'감사합니다(28.3%)', '엄마(17.8%)', '사랑해(12.1%)', '좋은(9.4%)', '사랑해요(9.4%)', '고마워(6.9%)', '사랑합니다(4.7%)',...",34,0.0062,"[김해숙님과 닮은 엄마 나를 아가라고 불러주는 엄마 오래오래 읽히는 좋은 이름 엄마 봤어요 감사합니다, 와명작 감사합니다, 좋은 감사합니다]"


,Topic Number,Weight,Comment
3400,Topic 1,0.8376,말그대로 미친 영화다
3242,Topic 1,0.8374,신기한 기법의 새로운 신선함 하나로 충분히 볼 가치가 기억상실에 새로운 접근법
1357,Topic 1,0.8374,인간이 태어나서 아들로써 남자로써아버지로써 남편으로써 살아가는건은힘들 고행길이 라는 생각이 드네요
3258,Topic 1,0.8371,흑백과 컬러의 교차 지점
2018,Topic 1,0.8371,여기가 천국인가요
5471,Topic 2,1.0000,기대보다 별로에요 끊키는 장면들
4314,Topic 2,1.0000,배우들의 연기는 그렇다쳐도 연출력의 허전함
3304,Topic 2,1.0000,곱씹으면서 봐야 흑백과 역재생의 의미와 목적이 이해되는
3305,Topic 2,1.0000,스토리의 영화는 처음 봐서 새롭고 재밌었어요 처음 봤을때는 이해가 안갔는데 해설 보니까 이해가 되네요
4309,Topic 2,1.0000,노년의 아름다운 사랑이야기


,Comment,Related Topic,Weight
0,29년 인생에 이렇게 여운이 진하게 남는 영화는 처음이에요 로맨스나 드라마 장르만 보면 코를 고는 저였지만 영화만큼은 처음부터 끝까지 몰입해서 봤습니다 연출과 ost가 눈물...,2,0.7903
1,지금이순간 무엇을 선택해야 삶에 끝에서 후회가 적을까요아이러니 하게도 죽음을 깊이 생각하니 지금 이순간 좋은 삶의 선택이 보이는듯 합니다 강추합니다,2,1.0000
2,지인이 괜찮다고 해서 기대감 없이 봤는데 생각보다 재밌어서 놀랐네요 여운이 오래 남아서 아직도 생각나네요,2,0.8776
3,치매 걸린 가족들이 겪어야 하는 리얼한 얘기였습니다,1,0.4147
4,아름다운 영화울기도 했지만 가족의 소중함을 있었던 영화입니다 치매 가족의 현실을 보여줘서 슬프지만 가족의 사랑 담아낸 영화라고 생각합니다 슬프지만 아름다운 영화제에서 51관...,1,0.8276
...,...,...,...
5467,많은걸 베웠습니다,1,0.2802
5468,진짜 흥행 안한건지 모르겠음 원래부터 서현진 배우님 팬이라서 영화도 기대 하고 봤지만 기대 이상으로 감동적이고 무엇보다 서현진배우님 연기에 한번 놀라고 감동받은 작품임 이거...,2,1.0000
5469,배우들 연기력 하나만으로 몰입갑 쩌는,1,0.5894
5470,서현진배우는 최고의 배우 입니다사랑합니다,2,1.0000


In [25]:
# 토픽 유사성 구조 시각화
model_BERT.visualize_topics()

In [37]:
# Topic 별 Hierarchy를 출력하여 대략적 Topic의 갯수를 가늠하기 용이함
model_BERT.visualize_hierarchy(top_n_topics=50)

In [38]:
# Topic을 대표하는 상위 단어 5개씩 출력
model_BERT.visualize_barchart(top_n_topics=50)

In [27]:
# Topic간 유사도를 Cosine Similarity로 계산후 Heatmap으로 표현
model_BERT.visualize_heatmap()

In [39]:
# 일자별로 어떤 토픽이 증가하고 줄어들었는지 트랜드 파악
topics_over_time = model_BERT.topics_over_time(docs=df['Comment'], timestamps=df.Date.to_list(), 
                                               global_tuning=True, evolution_tuning=True)
model_BERT.visualize_topics_over_time(topics_over_time, top_n_topics=50)

In [40]:
# Topic내 대표하는 단어들에 대해서 c-tf-idf로 계산해서 각 단어가 Topic에서 차지하는 중요도를 계산했던 것을 Rank 순서대로 출력
model_BERT.visualize_term_rank()